# VAEs Explained

- toc: true 
- badges: true
- comments: true
- categories: [generative models]

This blog is a collection of notes on the topic of VAE. Hope you find it useful.

#### Introduction: Latent Variable Models

Latent variable models are basically a class of generative models, where we try to model $p(x)$. The setting/assumptions under which they work are as follows,

1. A latent/hidden variable ($z\in\mathcal{Z}$) is responsible for generating Images ($x\in\mathcal{X}$)
2. It is easy to sample from the distribution $P(z)$ over $\mathcal{Z}$
3. A family of deterministic functions $f(z;\theta)$, parametrized by a vector $\theta$ , where $f:\mathcal{Z}\times \theta \rightarrow\mathcal{X}$

Basically, what this means is, if we sample $z$ from $P(z)$, and run it through $f(z;\theta)$ we should generate samples that look like they are from $P(X)$. These models aim to maximize to probability of each $X$ in the training set and therefore estimate parameters using maximum likelihood estimation given the generative process,

$$
P(X) = \int P(X|z;\theta)P(z)dz
$$

In VAEs specifically, 

- Output distribution $P(x|z)$  after running it through $f(z;\theta)$ (typically a neural network) is a gaussian distribution 
  $$
  P(x|z;\theta) \sim \mathcal{N}(X|f(z;\theta),\sigma^2I)
  $$
  where $\sigma$ is an hyperparameter

- Distribution over the latent variable $P(z) = \mathcal{N}(0, I)$ 

The idea is that if a latent structure exists, the neural network would be able to map the normally distributed $z$ to the latent values (For instance, the value of the digit, position in MNIST) in the first few layers and then use the later layers to generate images that have those latent values. 

Therefore, a way to estimate the $\theta$ in the above $P(x|z; \theta)$ is to,

1. Sample a number of $z$ values  
2. For each $z_i$ compute $f(z_i;\theta)$
3. Use the Monte Carlo Estimate to compute $P(X) = \frac{1}{n}\sum_iP(X|z_i)$ given the values from step 2

To maximize this $P(X)$, we can minimize the squared distance between $f(z)$ and $X$ (minimizing the negative log likelihood is equivalent to minimizing the squared euclidean distance between $f(z)$ and $X$ [[Proof](https://stats.stackexchange.com/questions/143705/maximum-likelihood-method-vs-least-squares-method/265430)]) and use gradient descent to estimate the parameters ( $\theta$ essentially the neural network weights). A visual description of the process is as follows,
<figure>
<img src="photos/vae-init.png" alt="vae-init" style="zoom:10%;" width="300" height="400" />
<figcaption> Fig 1. VAE as a Graphical Model </figcaption>
</figure>

**Why does this training strategy have a problem?** if we take an example of a generative model to predict the digit 2, Figure 2(a) is the target image (X) for which we are trying to find P(X) for. 

**Observation**: There is a stark difference between simple euclidean distance and perceptual distance. The prediction Figure 2(b) should be a worse prediction than Figure 2(c) (which is just shifting the digit towards down and right). However the contribution of Figure 2(b) in the likelihood would be more with respect to Figure 2(c) as the euclidean distance between the target and Figure 2(b) is 0.0387 while it is 0.2639 between the target and Figure 2(c). 

Two ways to avoid a situation like this, 

- One way could be to keep a small value of $\sigma$  in equation 2 (low value of $\sigma$ means [euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) is high unless it is very similar to $X$) so as to only allow very similar images to be generated. This means we would need a lot of $z$‘s to be able to generate something similar to $X$ and therefore contribute to likelihood ($P(X)$).
- Another solution is to change the similarity metric (which is euclidean distance currently) so as to generate more similar samples.
<figure>
<img src="photos/sampling-problem.png" alt="sampling-problem" style="zoom:25%;" width="500" height="200" />
<figcaption> Fig.2  It’s hard to measure the likelihood of images under a model using
only sampling. Given an image X (a), the middle sample (b) is much closer
in Euclidean distance than the one on the right (c). Because pixel distance is
so different from perceptual distance, a sample needs to be extremely close
in pixel distance to a datapoint X before it can be considered evidence that
X is likely under the model. Reference Paper: https://arxiv.org/pdf/1606.05908.pdf </figcaption>
</figure>

#### What can be done differently / what do VAEs do differently ?

Change the sampling strategy such that the sampled $z$ that are more likely to have produced $X$. Therefore, if we could sample from the true posterior distribution $p(z|X)$, we could assume we are generating $z$ that are likely to produce $X$. A slight problem that arises here, 
$$
p(z|X) = \frac{p(x|z;\theta)p(z)}{\int p(x|z';\theta)p(z')dz} = \frac{p(x|z;\theta)p(z)}{\int N(x|f(z';\theta), \sigma^2I)N(0, I)dz}
$$

The posterior is intractable as getting a [closed form expression](https://en.wikipedia.org/wiki/Closed-form_expression) for the denominator $\int N(x|f(z';\theta), \sigma^2I)N(0, I)$ in the above equation is not possible as the mean of the left term $f(z';\theta)$ is a neural network and computing a closed form expression using standard methods is difficult. Therefore, instead of drawing samples from $p(z|x)$, we draw from $q(z|x)$ which is an approximation of the true posterior. However, if we sample from an arbitrary distribution $q(z|x)$ and not from $N(0, I)$, how do we optimize $p(x)$?

We use the **variational lower bound (ELBO)** to optimize $P(X)$ while sampling $z$ from $q(z|x; \phi)$. There are different ways we can derive the ELBO to understand its need. Let’s go through them one by one.

**What’s $\phi$ in $q(z|x; \phi)$**? We generally use a neural network for $q(z|x; \phi)$. Therefore, $\phi$ are the neural network parameters that are used in the sampling process. 

#### Method 1: Maximizing the Log Likelihood ($\log P(X)$)

We can write $p(x)$ as the following expectation using joint probability and conditional probability,

$$
\log p(x) = \log \int p(x, z;\theta)dz  = \log \int p(x|z;\theta)p(z)dz  = \log \mathbb{E}_{z\sim p(z)}\left[p(x|z)\right]\\
$$

On dividing and multiplying with the approximate posterior distribution $q(z|x; \phi)$,

$$
\log p(x) = \log \int p(x, z;\theta)\frac{q(z|x; \phi)}{q(z|x; \phi)}dz = \log \left( \mathbb{E}_{z \sim q(z|x; \phi)}\left[ \frac{p(x, z;\theta)}{q(z|x; \phi)}\right]\right)
$$

The above term can be modified using Jensen’s Inequality ($\log \mathbb{E}(X) \geq \mathbb{E}(\log(X))$. It applies here since $\log$ is a concave function, read more [here](https://www.lri.fr/~sebag/COURS/EM_algorithm.pdf))

$$
\log p(x) =  \log \left( \mathbb{E}_{z \sim q(z|x; \phi)}\left[ \frac{p(x, z;\theta)}{q(z|x; \phi)}\right]\right) \geq \mathbb{E}_{z \sim q(z|x; \phi)}\left[\log \frac{p(x, z;\theta)}{q(z|x; \phi)} \right]
$$

$$
\log p(x) \geq \mathbb{E}_{z \sim q(z|x; \phi)}\left[\log \frac{p(x, z;\theta)}{q(z|x; \phi)} \right] \tag{1}
$$

The expectation term post Jensen’s Inequality on the right side is known as the **variational lower bound (ELBO)**. It is a lower bound on the $\log P(X)$ and the bound is tight when the approximated posterior $q(z|x)$ matches the true posterior $p(z|x)$ (replace the $q(z|x)$ term with $p(z|x)$ in the right hand side). 

If we try to break apart the **ELBO** term using Baye’s rule,

$$
\begin{aligned}
\mathbb{E}_{z \sim q(z|x; \phi)}\left[\log \frac{p(x, z;\theta)}{q(z|x; \phi)} \right]& = \mathbb{E}_{z \sim q(z|x; \phi)} \left[\log \frac{p(x|z;\theta)p(z)}{q(z|x; \phi)}\right]\\ & = \mathbb{E}_{z \sim q(z|x; \phi)} \left[\log p(x|z;\theta)\right] - \mathbb{E}_{z \sim q(z|x; \phi)}\left[\log \frac{q(z|x; \phi)}{p(z)}\right]
\end{aligned}
$$

The second term $\mathbb{E}_{z \sim q(z|x)}\left[\log \frac{q(z|x)}{p(z)}\right]$ is the KL divergence between $q(z|x)$ and $p(z)$. Therefore, we can write the ELBO objective as

$$
\mathbb{E}_{z \sim q(z|x; \phi)}\left[\log \frac{p(x, z;\theta)}{q(z|x; \phi)} \right] = \mathbb{E}_{z \sim q(z|x; \phi)} \left[\log p(x|z;\theta)\right] - D_{KL}(q(z|x; \phi) || p(z)) \tag{2}
$$

The above equation is important as it will serve as the objective function for variational autoencoders. Let’s park this here and look at another way to get the same equation for the Variational Lower Bound.

#### Method 2: Minimizing KL Divergence between $q(z|x; \phi)$ and $p(z|x)$

The main change that we bring about in a variational autoencoder is the approximation ($q(z|x; \phi)$) for the true posterior ($p(z|x)$) which we use for sampling. The KL divergence betweeen $q(z|x)$ and $p(z|x)$ is as follows,

$$
D_{KL}(q(z|x; \phi)||p(z|x)) = \mathbb{E}_{z \sim q(z|x; \phi)}\left[\log \frac{q(z|x; \phi)}{p(z|x)}\right]
$$
On using Baye’s rule to replace $p(z|x)$ as $\large\frac{p(x|z)p(z)}{p(x)}$ in the above equation,

$$
D_{KL}(q(z|x; \phi)||p(z|x)) = \mathbb{E}_{z \sim q(z|x; \phi)}\left[\log \frac{q(z|x; \phi)}{p(z|x)}\right] = \mathbb{E}_{z \sim q(z|x; \phi)}\left[\log \frac{q(z|x; \phi)p(x)}{p(x|z)p(z)}\right]
$$

We can remove the $p(x)$ term outside as it does not depend on $z$ and take it out of the expectation,

$$
\begin{aligned}
D_{KL}(q(z|x; \phi)||p(z|x)) &= \mathbb{E}_{z \sim q(z|x; \phi)}\left[\log \frac{q(z|x; \phi)p(x)}{p(x|z)p(z)}\right] \\&= \mathbb{E}_{z \sim q(z|x; \phi)}\left[\log \frac{q(z|x; \phi)}{p(x|z)p(z)}\right] + \log p(x)
\end{aligned}
$$

On rearranging the terms (take the expectation term from $RHS$ to $LHS$ and using property of logarithms),

$$
\log p(x) = \mathbb{E}_{z \sim q(z|x; \phi)} \left[\log p(x|z;\theta)\right] - \mathbb{E}_{z \sim q(z|x; \phi)}\left[\log \frac{q(z|x; \phi)}{p(z)}\right] + D_{KL}(q(z|x; \phi)||p(z|x))
$$

The first two terms on the RHS looks exactly same as the deconstructed ELBO term that we derived in equation 2. And as we know that the KL divergence metric is always greater than equal to 0, we can rewrite the above equation as,

$$
\log p(x) \geq \mathbb{E}_{z \sim q(z|x; \phi)} \left[\log p(x|z;\theta)\right] - D_{KL}(q(z|x; \phi) || p(z)) = ELBO(\theta, \phi)
$$

where equality holds when the approximated posterior $q(z|x)$ matches the true posterior $p(z|x)$ (Also seen in Equation 1).

#### Gradient Descent over ELBO

In summary, maximizing the ELBO objective (a lower bound on $\log p(x)$) will maximize the likelihood.

$$
ELBO() \rightarrow \mathbb{E}_{z\sim q_\phi(z)}\left[\log\frac{p_\theta(x, z)}{q_\phi(z|x) }\right]
$$

We basically need to find out $\theta$ and $\phi$ s.t

$$
\theta^*, \phi^* = \max_{\theta}\sum_{i}\max_{\phi}\mathbb{E}_{q_\phi}\left[\log\frac{p_\theta(x, z_i)}{q_\phi(z|x)}\right]
$$

The inner summation is over multiple $z_i$ sampled for each x. So we maximize over $\phi$ and then over $\theta$. We can look at this as a two step process,

$$
\phi_{i} \leftarrow \phi_{i} + \alpha \nabla_\phi ELBO(x_i, \theta, \phi_i) \\
\theta \leftarrow \theta + \beta \nabla_\theta \sum_i ELBO(x_i, \theta, \phi_i)
$$

The gradients wrt to $\theta$ can be computed easily as the only term that depends on $\theta$ is $p_\theta(x, z_i)$, where $\theta$ is the neural network weights.

$$
\nabla_\theta ELBO = \nabla_\theta\left[\mathbb{E}_{q_\phi}\log\frac{p_\theta(x, z_i)}{q_\phi(z|x)}\right]
$$

The gradient wrt to $\phi$ can be computed as,

$$
\begin{aligned}
\nabla_\phi ELBO & = \nabla_\phi\left[\mathbb{E}_{q_\phi}\log\frac{p_\theta(x, z_i)}{q_\phi(z|x)}\right] \\
&= \nabla_\phi\left[\int q_\phi(z|x)f_\phi(.)dz\right] \\
&= \int \nabla_\phi q_\phi(z|x)f_\phi(.) + q_\phi(z|x)\nabla_\phi f_\phi (.)dz \\
\end{aligned}
$$

If we use the [log derivative trick](https://math.stackexchange.com/questions/2554749/whats-the-trick-in-log-derivative-trick)  ($\nabla p(x) = p(x)\nabla \log p(x)$), for the first term, 

$$
\begin{aligned}
\nabla_\phi ELBO &= \int q_\phi(z|x)f_\phi(.)\nabla_\phi\log q_\phi(z|x) + q_\phi(z|x)\nabla_\phi f_\phi (.)dz \\
&= \int q_\phi(z|x)\left[f_\phi(.)\nabla_\phi\log q_\phi(z|x) + \nabla_\phi f_\phi (.)\right]dz \\
&= \mathbb{E}_{q_\phi}\left[\log\frac{p_\theta(x, z)}{q_\phi(z|x)}\nabla_\phi\log q_\phi(z|x) + \nabla_\phi f_\phi (.)\right] \\
\end{aligned}
$$

The second term can be removed by

$$
\begin{aligned}
\mathbb{E}_{q_\phi}\left[\nabla_\phi f_\phi (.)\right] &= \mathbb{E}_{q_\phi}\left[\nabla_\phi \log\frac{p_\theta(x, z)}{q_\phi(z|x)}\right]\\ &= \mathbb{E}_{q_\phi}\left[-\nabla_\phi \log q_\phi(z|x)\right]\\
&= -\int q_\phi(z|x)\nabla_\phi \log q_\phi(z|x) dz \\
&= -\int q_\phi(z|x) \frac{\nabla_\phi q_\phi(z|x)}{q_\phi(z|x)} dz \\
&= -\int \nabla_\phi q_\phi(z|x) dz \\
&= -\nabla_\phi \int q_\phi(z|x) dz = -\nabla_\phi \int q_\phi(z|x) dz = - \nabla_\phi (1)\\
&= 0
\end{aligned}
$$

Replace the above change for the right hand side term, 

$$
\nabla_\phi ELBO = \mathbb{E}_{q_\phi}\left[\log\frac{p_\theta(x, z)}{q_\phi(z|x)}\nabla_\phi\log q_\phi(z|x)  \right]
$$

The emperical form of ELBO can be written as,

$$
\text{Emperical Form : }\nabla_\phi \hat{ELBO} = \frac{1}{k}\sum_{i = 1}^{k}\left[\left(\log\frac{p_\theta(x, z)}{q_\phi(z|x)}\right)\nabla_\phi\log q_\phi(z|x)\right]
$$

Two conditions would be needed now to to compute this estimate,

1. We need to be able to sample $z$
2. The derivative of $ \log q_\phi(z|x)$ exists

Awesome! we now have a method to compute the gradient.

**Unfortunately! This poses a problem as well**

The gradient using this Monte Carlo estimate has a high variance and we need large number of samples before we can get a decent estimate. As our neural network $q_\phi(z|x)$ which generates a single $z$ after a forward pass with $x$, we would end up with the following issues.

1. To get multiple z’s is hard, as we are sampling one z per x through the neural network --> Computational Efficiency Problem
2. To get gradients on $\phi$, we would have to backpropogate through a stochastic node [Look at Fig 3] (Stochastic node as the node represents an entire distribution)
3. The estimate for the gradient has high variance (refer to this [excellently written blog](https://mpatacchiola.github.io/blog/2021/02/08/intro-variational-inference-2.html))

#### Reparametrization Trick

This trick was discussed in the paper [Autoencoding Variational Bayes](https://arxiv.org/abs/1312.6114). This proposes an alternative to the expensive sampling process and gives a way that allows for backpropogation through $\phi$. Instead of the neural network directly sampling from the distribution $q_\phi$, instead we sample the distribution statistics itself. Now the sampling process is simplified as follows, 

1. Neural Network $q_\phi$ predicts the mean $\mu$ and standard deviation $\sigma$ of a normal distribution taking input x
2. Sample multiple $\epsilon$ from a standard normal, $\mathcal{N}(0, I)$ -> Computationally Inexpensive 
3. To get $z$ from $\epsilon$, we use a deterministic differentiable function $T$ s.t. $z = T(\epsilon; \mu, \sigma)$

<figure>
<img src="photos/reparam_trick.png" alt="Reparameterization Trick" style="zoom:40%;" />
<figcaption> Fig 3. Reparametrization Trick : Image source: Slide 12 in Kingma’s NIPS 2015 workshop Link: http://dpkingma.com/wordpress/wp-content/uploads/2015/12/talk_nips_workshop_2015.pdf</figcaption>
</figure>

#### Training with ELBO

As we now have the objective function, we can now describe the specifics of the training process (see figure 4),

**1. Encoder for $q(z|x;\phi)$​**: 

- An encoder network helps in sampling $z$ from $q(z|x; \phi)$. The encoder instead of directly sampling $z$ , outputs the value of mean $\mu$ and variance $\Sigma$ of the distribution $q(z|x)$.
- The $\phi$ term in the encoder represents the parameters of the neural network.
- In order to now sample from this distribution $q(z|x; \phi)$ with the given mean and variance, we sample multiple $z$‘s from $N(0, I)$ and shift and scale them based on the mean and variance derived from the encoder [*Reparametrization Trick*]. 

**2. Decoder for $p(x|z; \theta)$**

- Given a $z$ from $q(z|x)$, we can try to get $p(x|z)$ using the decoder network $f(z;\theta)$. The decoder network is responsible for predicting the mean of $p(x|z; \theta) = N(x | f(z;\theta), \sigma^2I)$

**3. Loss function (ELBO Term)**

The ELBO objective is maximized to train the model. The objective is as follows,

$$
ELBO(\theta, \phi) = \mathbb{E}_{z \sim q(z|x)} \left[\log p(x|z;\theta)\right] - D_{KL}(q(z|x; \phi)\: ||\: p(z))
$$

The first term can be seen as the reconstruction error term (maximizing ELBO will minimize the reconstruction loss),  

$$
\mathbb{E}_{z \sim q(z|x)} \left[\log p(x|z;\theta)\right] \sim \frac{1}{m}\sum_j-\left[\frac{x - f(z_j;\phi)}{2\sigma^2}\right]^2
$$

The second term can be seen as KL to the prior (maximizing ELBO will decrease the distance between variational approximated posterior $q(z|x; \phi)$ and $p(z)$). As both $q(z|x; \phi)$ and $p(z)$ are gaussians, the KL term has a [closed form expression](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence#Multivariate_normal_distributions).

The following image depicts the entire training pipeline of a VAE.

<figure>
<img src="photos/vae-fn.png" alt="Screenshot 2022-02-21 at 1.27.08 PM" width="800" />
<figcaption> Fig 4. Training a VAE </figcaption>
</figure>

#### References

1. [Tutorial 5: variational autoencoders](https://www.borealisai.com/en/blog/tutorial-5-variational-auto-encoders/)
2. [The Expectation Maximization Algorithm A short tutorial](https://www.lri.fr/~sebag/COURS/EM_algorithm.pdf)
3. [Tutorial on Variational Autoencoders](https://arxiv.org/pdf/1606.05908.pdf)
4. [Why does the reparametrization trick work?](https://stats.stackexchange.com/questions/199605/how-does-the-reparameterization-trick-for-vaes-work-and-why-is-it-important)